<a href="https://colab.research.google.com/github/seon03/MachineLearning/blob/main/feature_final_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. mon_standard.pkl > array code



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file
print("Loading datafile...")
with open("/content/drive/MyDrive/mon_standard.pkl", 'rb') as fi: #Colab 에서 mon_standard.pkl 파일의 경로
    data = pickle.load(fi)

# X1 ~ X4: Continuous features(4개)
X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
y = []  # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]

X3 = [] # Array to store bursts - 19,000 instances
X4 = [] # Array to store cumulative packet sizes - 19,000 instances

#X5 ~ X17: Categorical features(13개)
         # Arrays to store ~
X5 = []  # rank 1.Number of incoming packets
X6 = []  # rank 2.Number of outgoing packets as a fraction of the total number of packets
X7 = []  # rank 3.Number of incoming packets as a fraction of the total number of packets
X8 = []  # rank 4.Standard deviation of the outgoing packet ordering list
X9 = []  # rank 5.Number of outgoing packets
X10 = [] # rank 6.Sum of all items in the alternative concentration feature list
X11 = [] # rank 7.Average of the outgoing packet ordering list
X12 = [] # rank 8.Sum of incoming, outgoing and total number of packets
X13 = [] # rank 9.Sum of alternative number packets per second
X14 = [] # rank 10.Total number of packets
         # X15 ~ X17: Packet concentration and ordering features list
X15 = [] # rank 11.Mean of the number of outgoing packets in chunks of 20 packets
X16 = [] # rank 12.Standard deviation of the incoming packet ordering list
X17 = [] # rank 13.Average of the incoming packet ordering list


# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.

    for sample in data[i]:
        time_seq = [] #X1
        size_seq = [] #X2
        size_burst_seq = [] #X3
        size_cumulative_seq = [] #X4
        packet_sum = [] #X12
        incoming_count = 0 #X5
        outgoing_count = 0 #X9
        outgoing_fraction = 0.0 #X6
        incoming_fraction = 0.0 #X7
        outgoing_packet_order = [] #X8
        incoming_packet_order = [] #X16
        outgoing_packet_size_order=[]
        split_outgoing_packet_order = [] #X15



        burst_sum = 0
        last_sign = None
        cumulative_sum = 0

        for c in sample:
            time_seq.append(abs(c)) # Append timestamp to the timestamp sequence X1

            # Calculate the direction and size for packet size
            dr = 1 if c > 0 else -1
            packet_size = dr * 512
            size_seq.append(packet_size)  # Append direction * 512 to the packet size sequence


           # X3
           # Calculate the burst sequence based on the sign of the value
            if c > 0:
                sign = 1
                outgoing_count += 1 #X9 (rank 5)
                outgoing_packet_order.append(abs(c))
                #X15
                outgoing_packet_size_order.append(packet_size)
                #split_outgoing_packet_order = np.array(outgoing_packet_size_order).reshape(20,1)
                split_outgoing_packet_order = [outgoing_packet_size_order[i * 20:(i + 1) * 20] for i in range((len(outgoing_packet_size_order) + 20 - 1) // 20 )]# 20개씩 분할하기

            else:
                sign = -1
                incoming_count += 1 #X5 (rank 1)
                incoming_packet_order.append(c)

            if last_sign is None:  # For the first value
                burst_sum = packet_size
            elif sign == last_sign:  # If the sign remains the same, add to the burst
                burst_sum += packet_size
            else:  # If the sign changes, append the accumulated burst and start a new one
                size_burst_seq.append(burst_sum)
                burst_sum = packet_size

            last_sign = sign

            #X4
            cumulative_sum += packet_size
            size_cumulative_seq.append(cumulative_sum)


        #X8 (rank 4)
        std_dev_outgoing_order = np.std(outgoing_packet_order)


        #X6 (rank 2), X7 (rank 3)
        total_packets = len(sample)
        if total_packets > 0:
            outgoing_fraction = outgoing_count / total_packets #X6
            incoming_fraction = incoming_count / total_packets #X7




        # X11(rank 7)
        avg_outgoing_packet_order = np.average(outgoing_packet_order)

        # X12(rank 8)
        packet_sum = outgoing_count + incoming_count + total_packets

        # X13(rank 9)
        # Calculate the sum of alternative number packets per second


        # X14(rank 10)
        total_packets_num = len(sample)

        #X15
        mean_split_outgoing_packet = np.mean(np.mean(split_outgoing_packet_order, axis=0))

        # X16(rank 12)
        std_dev_incoming_order = np.std(incoming_packet_order)

        # X17(rank 13)
        avg_incoming_packet_order = np.average(incoming_packet_order)


        X1.append(time_seq)
        X2.append(size_seq)
        X3.append(size_burst_seq)
        X4.append(size_cumulative_seq)
        X5.append(incoming_count)
        X6.append(outgoing_fraction)
        X7.append(incoming_fraction)
        X8.append(std_dev_outgoing_order)
        X9.append(outgoing_count)
        #X10.append()
        X11.append(avg_outgoing_packet_order)
        X12.append(packet_sum)
        #X13.append()
        X14.append(total_packets_num)
        X15.append(mean_split_outgoing_packet)
        X16.append(std_dev_incoming_order)
        X17.append(avg_incoming_packet_order)
        y.append(label)

size = len(y)

print(f'Total samples: {size}') # Output: 19000
# Extract y values and remove duplicates
unique_y = list(set(y))


Loading datafile...


/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


Total samples: 200

Sequence of Timestamps (X1) - First 5 samples:
Sample 0: [0.0, 0.14, 0.14, 0.31, 0.31, 0.51, 0.51, 0.51, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.88, 0.89, 1.13, 1.13, 1.3, 1.3, 1.3, 1.3, 1.3, 1.47, 1.47, 1.58, 1.72, 1.72, 1.72, 1.97, 2.21, 2.21, 2.21, 2.38, 2.38, 2.38, 2.38, 2.38, 2.38, 2.38, 2.47, 2.47, 2.48, 2.68, 2.68, 2.68, 2.98, 2.98, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.05, 3.08, 3.08, 3.08, 3.08, 3.08, 3.08, 3.08, 3.08, 3.08, 3.08, 3.08, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.15, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.17, 3.19, 3.48, 3.48, 3.48, 3.48, 3.48, 3.48, 3.49, 3.68, 3.69, 3.69, 3.69, 3.69, 3.69, 3.69, 3.69, 3.69, 3.69, 3.71, 3.71

2. unmon_standard10.pkl > array code

In [ ]:
import pickle

TOTAL_URLS = 10000  # total number in the dataset

# Load 10,000 unmon pickle file
print("Loading datafile...")
with open('/content/drive/MyDrive/unmon_standard10.pkl', 'rb') as f:  # Path to unmon_standard10.pkl in Colab
    x = pickle.load(f)

size = len(x)
print(f'Total samples: {size}')

X1 = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information

for i in range(TOTAL_URLS):
    size_seq = []
    time_seq = []
    for c in x[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.
    X1.append(time_seq)
    X2.append(size_seq)

print(len(X1)) # Print the length of X1

# Print the first 10 entries in X1, X2, and y
for i in range(9990, 9999):
    print(f"Sample {i + 1}:")
    print("X1:", X1[i])
    print("X2:", X2[i])
    #print("y:", y[i])
    print("-" * 20)

Loading datafile...
Total samples: 10000
10000
Sample 9991:
X1: [0.0, 0.1, 0.1, 0.24, 1.06, 1.21, 1.21, 1.35, 1.35, 1.35, 1.35, 1.35, 1.35, 1.35, 1.36, 1.36, 1.36, 1.37, 1.37, 1.37, 1.37, 1.44, 2.58, 2.58, 2.7, 2.7, 2.7, 2.75, 2.78, 2.78, 2.89, 2.89, 2.89, 2.89, 2.89, 2.89, 2.89, 2.9, 2.9, 2.9, 2.9, 2.9, 2.9, 2.9, 2.9, 2.95, 2.95, 2.96, 2.96, 2.96, 2.96, 2.96, 2.96, 2.96, 2.98, 2.98, 2.98, 2.98, 2.98, 2.98, 2.98, 3.06, 3.06, 3.06, 3.11, 3.12, 3.12, 3.12, 3.12, 3.12, 3.12, 3.12, 3.13, 3.13, 3.13, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.14, 3.18, 3.18, 3.18, 3.18, 3.18, 3.22, 3.22, 3.22, 3.22, 3.37, 3.42, 3.42, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.58, 3.58, 3.58, 3.58, 3.58, 3.58, 3.58, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.61, 3.61, 3.64, 3.64, 3.64, 3.66, 3.66, 3.66, 3.68, 3.72, 3.72, 3.72, 3.72, 3.72, 3.74, 3.74, 3.74, 3.74, 3.74, 3.74, 3.74, 3.74, 3.74, 3.82, 3.83, 3.83, 3.92, 3.92, 3.92, 3.92, 3.92, 3.92,